In [2]:
import deepsmiles
import mbuild as mb
import numpy as np

# poly-smiles:
Quickly and easily create SMILES strings of polymers of any length from the SMILES string of the monomer

The key to this functionality is the use of **DeepSMILES**, which has more simple formatting rules that typically required of SMILES strings.

**Repo**: https://github.com/baoilleach/deepsmiles

`pip install --upgrade deepsmiles`

### Current Functionality:
    - Create polymer smiles string from a monomer
    - Creates polydisperse system from a user created dictionary
    
### Planned Functionality:
    - Create polydisperse system from a distribution function
    - Create a polydisperse system that corresponds with a PDI value
    - Create polymers with different regio-regularities
    - Specify and add end groups
    
    
### To Do:
    - Change atom counting logic to handle elements with 2 characters
            - For example, a monomer with Cl won't work with poly_smiles function
            
    - See if bond site can be indicated by string index # rather than **
        Does the index # translate from SMILES to DeepSMILES?
        
    - Run some quick performance checks
        - How does it scale with num of atoms?
        - Compare against mbuild's current polymer plugin
        - Compare against initializing a system of just monomers
            - Should indicate how much of the time required is due to the polymerization step
            
     - Need to create better handling of mb box. User specifices a density, or maybe just start with very large box, pack it and shrink it.

In [3]:
def convert_smiles(smiles=False, deep=False):   
    '''
    smiles and deep must be str format
    Converts from SMILES to DeepSMILES and vice versa.
    Whichever has a string provided, will convert to the other.
    If strings are proivded for both, then nothing happens
    '''
    
    converter = deepsmiles.Converter(rings=True, branches=True)
    if smiles and deep:
        print('Only provide a string for one of smiles or deep')
        return()
    if smiles: # Convert from SMILES to DeepSMILES
        deep_string = converter.encode(smiles)
        return deep_string
    if deep: # Convert from DeepSMILES to SMILES
        smiles_string = converter.decode(deep)
        return smiles_string 
    
    
def poly_smiles(monomer_string, length=2, ftype = 'mol2', string_only=True,
                energy_min=False, save=False, visualize=False):
    
    # Find how many branch-brackets are required at polymerization site
    atom_count = 0
    bracket_count = 0
    for s in monomer_string:
        if s.isalpha():
            atom_count += 1    
        if s == ')':
            bracket_count += 1 
    if bracket_count == 0:
        brackets = ')' * atom_count
    elif bracket_count != 0:
        brackets = ')' * (atom_count - bracket_count)
       
    # Find index num of poly site on modified DEEP SMILES string
    monomer_list = list(monomer_string)
    if '*' not in monomer_list:
        return(print('ERROR: Identify the wanted polymerization site using *x*'))
    key_indices = [index for index, value in enumerate(monomer_list) if value == '*']  
    if len(key_indices) != 2:   # Checks for only a single given poly site
        return(print('ERROR: Select only one polymerization site using *x*'))
    if key_indices[1] - key_indices[0] != 2:   # Check that the * are surrounding only a single atom
        return(print('ERROR: Select only one polymerization site using *x*'))
    monomer_list[key_indices[1]] = '{}' + '{}'.format(brackets) # Create poly site+brackets to the right of the atom
    monomer_list.remove('*')
    template = ''.join(monomer_list)  # Monomer string with the needed {} and without second * in the string
    monomer_list.remove('{}' + '{}'.format(brackets))
    monomer = ''.join(monomer_list)  # Pure deepsmiles monomer string without {} or *
    # What is the difference between monomer and monomer_string without **??
    
    # Loop & format polymer
    polymer = '{}'
    for i in range(0, length):
        if i == length - 1:
            polymer = polymer.format(monomer)
            break
        polymer = polymer.format(template)

    polymer_smiles = convert_smiles(deep = polymer)
    compound = mb.load(polymer_smiles, smiles = True)
    if string_only:
        return(polymer_smiles)
    if energy_min:
        compound.energy_minimize(steps = 25, forcefield='GAFF', algorithm = 'md')
    if visualize:
        compound.visualize().show()  
    if save:
        file_name = "comp_{}mer.{}".format(length, ftype)
        compound.save(file_name, overwrite = True)
    return compound


def viz(smiles_string, deep = True, energy_min = False):
    '''
    Lets you quickly visualize a smiles or deepsmiles string
    '''
    if deep:
        smiles_string = convert_smiles(deep = smiles_string)
    comp = mb.load(smiles_string, smiles = True)
    if energy_min:
        comp.energy_minimize(algorithm='md', steps=100)
    comp.visualize().show()
    
    
def polydisperse_system(system_dict=None, distribution=None, pdi=None):
    
    system_compounds = []
    if system_dict:
        for monomer in system_dict:
            for length in system_dict[monomer]:
                num = system_dict[monomer][length]
                polymer = poly_smiles(monomer, length, string_only=False)
                for i in range(num):
                    system_compounds.append(polymer)
        system = mb.fill_box(system_compounds, [1 for i in system_compounds], density=100)
        return system
            
    if distribution:
        pass
    
    if pdi:
        pass
        

In [38]:
#poly_smiles needs a smiles string in the DeepSmiles format; use this to convert
p3ht = 'c1scc(CCCCCC)c1'
p3ht_deep = convert_smiles(smiles=p3ht)
print(p3ht_deep)
viz(p3ht_deep)

csccCCCCCC))))))c5


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [39]:
# Use poly_smiles function to try out different polymerization sites, visualize to check outcome

p3htmer = poly_smiles('cs*c*cCCCCCC))))))c5', length=10, string_only=False, visualize=True)

/home/chris/cme/forks/mbuild/mbuild/compound.py:2746: UserWarning: No unitcell detected for pybel.Molecule c1sc(c2sc(c3sc(c4sc(c5sc(c6sc(c7sc(c8sc(c9sc(c%10scc(CCCCCC)c%10)c(CCCCCC)c9)c(CCCCCC)c8)c(CCCCCC)c7)c(CCCCCC)c6)c(CCCCCC)c5)c(CCCCCC)c4)c(CCCCCC)c3)c(CCCCCC)c2)c(CCCCCC)c1	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [36]:
# Generate a system with 3 different sizes of p3ht:

p3ht = 'cs*c*cCCCCCC))))))c5' # Specify polymerization site with **
p3ht_d = {4:3, 6:8, 8:2}  # polymer_length:number_of_polymers
system_d = {p3ht:p3ht_d}

system = polydisperse_system(system_dict=system_d)
system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [35]:
# Creating a mixture:

benzene = 'cc*c*ccc6'  # Specify polymerization site with **
p3ht = 'cs*c*cCCCCCC))))))c5'
benzene_d = {4:1, 6:1, 8:1}  # polymer_length:number_of_polymers
p3ht_d = {4:1, 6:1, 8:1}
system_d = {benzene:benzene_d,
           p3ht:p3ht_d}

system = polydisperse_system(system_dict=system_d)
system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [21]:
ptb7 = 'CCCCC(CC)COC(=O)c1sc2c(scc2c1F)c3sc4c(OCC(CC)CCCC)c5ccsc5c(OCC(CC)CCCC)c4c3'
ptb7_deep = convert_smiles(smiles=ptb7)
print(ptb7_deep)

CCCCCCC))COC=O)csccscc5c8F)))))csccOCCCC))CCCC)))))))cccsc5cOCCCC))CCCC)))))))c9c%12


In [23]:
ptb7_deep='ccccOCCCC))CCCC)))))))cscccscCOCCCCCC))))CC)))))=O))cF)c5cs8))))))))cc5cOCCCC))CCCC)))))))c9s%12'
viz(ptb7_deep)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [29]:
poly_smiles('ccccOCCCC))CCCC)))))))cscccscCOCCCCCC))))CC)))))=O))cF)c5*c*s8))))))))cc5cOCCCC))CCCC)))))))c9s%12',
           string_only=False,
           visualize=True,
           length=15)

/home/chris/miniconda3/envs/mosdev/lib/python3.7/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/home/chris/cme/forks/mbuild/mbuild/compound.py:2746: UserWarning: No unitcell detected for pybel.Molecule c1cc2c(OCC(CC)CCCC)c3sc(c4c5sc(C(=O)OCC(CCCC)CC)c(F)c5c(c5cc6c(OCC(CC)CCCC)c7sc(c8c9sc(C(=O)OCC(CCCC)CC)c(F)c9c(c9cc%10c(OCC(CC)CCCC)c%11sc(c%12c%13sc(C(=O)OCC(CCCC)CC)c(F)c%13c(c%13cc%14c(OCC(CC)CCCC)c%15sc(c%16c%17sc(C(=O)OCC(CCCC)CC)c(F)c%17c(c%17cc%18c(OCC(CC)CCCC)c%19sc(c%20c%21sc(C(=O)OCC(CCCC)CC)c(F)c%21c(c%21cc%22c(OCC(CC)CCCC)c%23sc(c%24c%25sc(C(=O)OCC(CCCC)CC)c(F)c%25c(c%25cc%26c(OCC(CC)CCCC)c%27sc(c%28c%29sc(C(=O)OCC(CCCC)CC)c(F)c%29c(c%29cc%30c(OCC(CC)CCCC)c%31sc(c%32c%33sc(C(=O)OCC(CCCC)CC)c(F)c%33c(c%33cc%34c(OCC(CC)CCCC)c%35sc(c%36c%37sc(C(=O)OCC(CCCC)CC)c(F)c%37c(c%37cc%38c(OCC(CC)CCCC)c%3

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

< 1547 particles, non-periodic, 1621 bonds, id: 139740032892688>

In [30]:
poly_smiles('ccccOCCCC))CCCC)))))))cscccscCOCCCCCC))))CC)))))=O))cF)c5*c*s8))))))))cc5cOCCCC))CCCC)))))))c9s%12',
           length=15)

'c%75cc%74c(OCC(CC)CCCC)c%73sc(c%72c1sc(C(OCC(CCCC)CC)=O)c(F)c1c(c%71cc%70c(OCC(CC)CCCC)c%69sc(c%68c2sc(C(OCC(CCCC)CC)=O)c(F)c2c(c%67cc%66c(OCC(CC)CCCC)c%65sc(c%64c3sc(C(OCC(CCCC)CC)=O)c(F)c3c(c%63cc%62c(OCC(CC)CCCC)c%61sc(c%60c4sc(C(OCC(CCCC)CC)=O)c(F)c4c(c%59cc%58c(OCC(CC)CCCC)c%57sc(c%56c5sc(C(OCC(CCCC)CC)=O)c(F)c5c(c%55cc%54c(OCC(CC)CCCC)c%53sc(c%52c6sc(C(OCC(CCCC)CC)=O)c(F)c6c(c%51cc%50c(OCC(CC)CCCC)c%49sc(c%48c7sc(C(OCC(CCCC)CC)=O)c(F)c7c(c%47cc%46c(OCC(CC)CCCC)c%45sc(c%44c8sc(C(OCC(CCCC)CC)=O)c(F)c8c(c%43cc%42c(OCC(CC)CCCC)c%41sc(c%40c9sc(C(OCC(CCCC)CC)=O)c(F)c9c(c%39cc%38c(OCC(CC)CCCC)c%37sc(c%36c%10sc(C(OCC(CCCC)CC)=O)c(F)c%10c(c%35cc%34c(OCC(CC)CCCC)c%33sc(c%32c%11sc(C(OCC(CCCC)CC)=O)c(F)c%11c(c%31cc%30c(OCC(CC)CCCC)c%29sc(c%28c%12sc(C(OCC(CCCC)CC)=O)c(F)c%12c(c%27cc%26c(OCC(CC)CCCC)c%25sc(c%24c%13sc(C(OCC(CCCC)CC)=O)c(F)c%13c(c%23cc%22c(OCC(CC)CCCC)c%21sc(c%20c%14sc(C(OCC(CCCC)CC)=O)c(F)c%14c(c%19cc%18c(OCC(CC)CCCC)c%17sc(c%16c%15sc(C(OCC(CCCC)CC)=O)c(F)c%15cs%16)cc%17c(OCC(